# Machine Learning. Caso: Reconocimiento de Dígitos

Instrucciones:
* Por favor, complete las tareas de este notebook. Deberá enviar este notebook, así como una versión en PDF, a la plataforma Univirtual.
* Para crear el PDF, vaya a Archivo > Descargar como. Puede exportar a PDF mediante Latex, o exportar a HTML y luego imprimir a PDF.
* Añada una explicación clara de su enfoque y una interpretación detallada de sus resultados para cada subpregunta. Para ello, utilice celdas Markdown.

* Añada a continuación los nombres de todos los miembros del equipo.

**Integrantes:**

1.   Integrante 1:
2.   Integrante 2:

## MNIST

Se va a usar para esta práctica la base de datos [MNIST](https://www.openml.org/d/554)  de dígitos manuscritos que cuenta con  784 características (datos brutos disponibles en: http://yann.lecun.com/exdb/mnist/9). Esta base de datos contiene 70.000 imágenes de dígitos escritos a mano, clasificados en 10 tipos de dígitos (del 0 al 9), cada uno representado por valores de 28 por 28 píxeles. Puede descargarlo de OpenML y visualizar algunos de los ejemplos

In [ ]:
!pip install preamble

In [ ]:
!pip install openml

In [ ]:
# Imports generales
%matplotlib inline
from preamble import *
import matplotlib.pyplot as plt
plt.rcParams['savefig.dpi'] = 100 # Esto controla el tamanio de tus imagenes

In [ ]:
import openml as oml

In [ ]:
# Descarga de la data de MINST. Demora un poco la primera vez
mnist = oml.datasets.get_dataset(554)
X, y, _, _ = mnist.get_data(target=mnist.default_target_attribute);
mnist_classes = {0:"Cero", 1: "Uno", 2: "Dos", 3: "Tres", 4: "Cuatro", 5: "Cinco",
                  6: "Seis", 7: "Siete", 8: "Ocho", 9: "Nueve"}

In [ ]:
X

In [ ]:
# Toma algunos ejemplos aleatorios, reescala las images al tamaño de 32x32 image y muestralas
from random import randint
fig, axes = plt.subplots(1, 5,  figsize=(10, 5))
for i in range(5):
    n = randint(0,70000)
    axes[i].imshow(X.values[n].reshape(28, 28), cmap=plt.cm.gray_r)
    axes[i].set_xlabel((mnist_classes[int(y.values[n])]))
    axes[i].set_xticks(()), axes[i].set_yticks(())
plt.show();

1_ Evalúe k-Nearest Neighbors, Regresión Logística y SVM Lineal. (2.5 puntos)
- Tome unav*submuestra estratificada* del 10% de los datos. Use esta muestra para todos los pasos siguientes.
- Evalúe los 3 clasificadores con sus configuraciones por defecto. Use cross-validation con 3 folds, muestre el accuracy y la desviación estándar.
- Discuta cuál funciona mejor.

Nota: puede utilizar una muestra más pequeña durante las pruebas y la corrección de errores.

2_ Ajuste los parámetros usando GridSearchCV. (2.5 puntos)
- Varíe el hiperparámetro principal (C o k) para los 3 modelos. ¿Mejoran los resultados?  ¿Qué rangos influyen en el rendimiento?
- Visualice el score de prueba y el score de entrenamiento en función de los parámetros de los tres modelos. Discuta cuándo (para qué valores) del modelo se produce underfitting u overfitting.

Nota: Puede utilizar un line plot o 1D heatmap  para la visualización. Utilice de nuevo la validación cruzada (3-fold cross-validation) y el accuracy.

In [ ]:
# Puede utilizar este gráfico genérico para  la búsqueda en 1D (1D grid search)
# grid_search: el resultado del GridSearchCV
# param_name: el nombre del parámetro que se está variando
def plot_tuning(grid_search, param_name):
    plt.figure()
    plt.plot(grid_search.param_grid[param_name], grid_search.cv_results_['mean_test_score'], marker = '.', label = 'Test score')
    plt.plot(grid_search.param_grid[param_name], grid_search.cv_results_['mean_train_score'], marker = '.', label = 'Train score')
    ax = plt.gca()
    ax.set_ylabel('score (ACC)')
    ax.set_xlabel(param_name)
    ax.legend()
    plt.title(grid_search.best_estimator_.__class__.__name__)
    print('Best configuration:' + str(grid_search.best_params_))
    print('Best score (ACC):' + str(grid_search.best_score_))

3_ Analice qué tipos de clasificaciones erróneas (misclassifications) se cometen (2.5 puntos)
- Cree una división estándar train_test
- Entrene la Regresión Logística en los datos de entrenamiento y genere predicciones en el conjunto de prueba
- Visualice (como arriba) unos cuantos ejemplos mal clasificados por Regresión Logística. Discusión: ¿son realmente casos difíciles?
- Construya la matriz de confusión ('confusion_matrix') de todas las predicciones. Discuta en qué clases comunmente se produce confusión/errores.

4_ Visualice los parámetros del modelo (2 puntos)
- Recupere todos los parámetros del modelo (coefficients) para la Regresión Logística y Linear Support Vector Machines
- Plot los coeficientes en una imagen de 28*28 como en el caso anterior.
- Interprete los resultados. ¿A qué píxeles prestan más atención los modelos? ¿Por qué? ¿Hay alguna diferencia entre ambos modelos?

5_ Ahora use el dataset CIFAR-10  (Alex Krizhevsky (2009) Learning Multiple Layers of Features from Tiny Images, Tech Report.).  (2.5 puntos)

 [CIFAR-10](https://www.openml.org/d/40927)  es un subconjunto etiquetado del conjunto de datos de 80 millones de imágenes diminutas. Consta (originalmente) de 32x32 imágenes en color que representan 10 clases de objetos:

0.   avión
1.   automóvil
2.   pájaro
3.   gato
4.   ciervo
5.   perro
6.   rana
7.   caballo
8.   barco
9.   camión

CIFAR-10 contiene 6000 imágenes por clase. La división original de entrenamiento-prueba las dividió aleatoriamente en 5000 imágenes de entrenamiento y 1000 de prueba por clase.


*   ¿Cómo aplicarías en este dataset los procesos previoes al paso 1 de este notebook (es decir, la lectura, el pre-tratamiento y el plot), ¿qué ajustes  deberías hacer a las imágenes?. Justifique su respuesta.

*   ¿Cómo aplicarías los pasos del punto 1 de este notebook (es decir Evalúe k-Nearest Neighbors, Regresión Logística y SVM Lineal). Muestre sus resultados, coméntelos y explique las adecuaciones que ha tenido que realizar.

*   Puede usar como base el código que se presenta en las líneas siguientes:



In [ ]:
# Descargar la data para el dataset CIFAR-10. Demora un poco la primera vez
cifar = oml.datasets.get_dataset(40927)
X_ori, y_ori, _, _ = cifar.get_data(target=cifar.default_target_attribute);
cifar_classes = {0:"airplane", 1: "automobile", 2: "bird", 3: "cat", 4: "deer", 5: "dog",
                  6: "frog", 7: "horse", 8: "ship", 9: "truck"}



In [ ]:
X_ori

Ajuste el código de las líneas siguientes para que funcione de manera correcta para pintar las imágenes en escala de grises (grayscale).  Cambie lo que considere necesario.

In [ ]:
# Toma algunos ejemplos aleatorios, reescala las images al tamaño de 32x32 image y muestralas
from random import randint
fig, axes = plt.subplots(1, 5,  figsize=(10, 5))
for i in range(5):
    n = randint(0,60000)
    axes[i].imshow(X_ori.values[n].reshape(32, 32), cmap=plt.cm.gray_r)
    axes[i].set_xlabel((cifar_classes[int(y_ori.values[n])]))
    axes[i].set_xticks(()), axes[i].set_yticks(())
plt.show();